In [13]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
import warnings
import copy
from scipy.stats import skew, kurtosis, median_abs_deviation, linregress
from scipy.signal import find_peaks, lombscargle
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, roc_curve, auc
from sklearn.inspection import permutation_importance

# Xử lý tương thích phiên bản scipy cho hàm tích phân
try:
    from scipy.integrate import trapezoid as trapz
except ImportError:
    try:
        from scipy.integrate import trapz
    except ImportError:
        from numpy import trapz

# Cấu hình môi trường
warnings.filterwarnings('ignore')
BASE_PATH = "/kaggle/input/astronomical"
TRAIN_LOG_PATH = f"{BASE_PATH}/train_log.csv"

In [14]:
# ==========================================
# CÁC HÀM HỖ TRỢ TÍNH TOÁN (STATISTICS & PHYSICS)
# ==========================================

def calculate_peak_lags(band_max_times):
    """Tính độ trễ thời gian giữa đỉnh sáng của các băng tần."""
    lags = {}
    pairs = [('u', 'g'), ('g', 'r'), ('r', 'i')]
    
    for b1, b2 in pairs:
        if (b1 in band_max_times) and (b2 in band_max_times):
            lag = band_max_times[b1] - band_max_times[b2]
            lags[f'lag_{b1}_{b2}'] = lag
        else:
            lags[f'lag_{b1}_{b2}'] = 0
    return lags

def calculate_color_slope(time, flux, bands):
    """Tính độ dốc thay đổi màu (g - r) theo thời gian."""
    try:
        mask_g = (bands == 'g')
        mask_r = (bands == 'r')
        if (np.sum(mask_g) < 2) or (np.sum(mask_r) < 2): 
            return 0
        
        df_g = pd.DataFrame({'t': time[mask_g], 'g': flux[mask_g]}).sort_values('t')
        df_r = pd.DataFrame({'t': time[mask_r], 'r': flux[mask_r]}).sort_values('t')
        
        # Merge dữ liệu g và r với sai số thời gian cho phép là 1 ngày
        df_merged = pd.merge_asof(df_g, df_r, on='t', tolerance=1, direction='nearest').dropna()
        
        if len(df_merged) < 3: 
            return 0
        
        df_merged['color'] = df_merged['g'] - df_merged['r']
        slope, _, _, _, _ = linregress(df_merged['t'], df_merged['color'])
        return slope
    except:
        return 0

def calculate_advanced_stats(flux, flux_err):
    """Tính các chỉ số thống kê mô tả phân phối flux."""
    if len(flux) < 2: 
        keys = ['iqr', 'variability_index', 'norm_mad', 'excess_variance', 'flux_ratio_95_5', 'beyond_1std']
        return {k: 0 for k in keys}

    flux = np.nan_to_num(flux, nan=np.nanmedian(flux))
    
    q75, q25 = np.percentile(flux, [75, 25])
    iqr = q75 - q25
    std_val = np.std(flux)
    mean_val = np.mean(flux)
    med_val = np.median(flux)
    
    var_index = (np.max(flux) - np.min(flux)) / (std_val + 1e-9)
    
    mad_val = median_abs_deviation(flux)
    norm_mad = mad_val / (np.abs(med_val) + 1e-9)
    
    mean_sq_err = np.mean(flux_err**2)
    excess_var = (np.var(flux) - mean_sq_err) / (mean_val**2 + 1e-9)
    
    q95, q05 = np.percentile(flux, [95, 5])
    flux_ratio_95_5 = q95 / (abs(q05) + 1e-9)
    
    beyond_1std = np.sum(np.abs(flux - mean_val) > std_val) / len(flux)
    
    return {
        'iqr': iqr,
        'variability_index': var_index,
        'norm_mad': norm_mad,
        'excess_variance': max(0, excess_var),
        'flux_ratio_95_5': flux_ratio_95_5,
        'beyond_1std': beyond_1std
    }

def calculate_variability_features(flux, time):
    """Tính các đặc trưng biến thiên chuỗi thời gian (Von Neumann, Periodogram)."""
    if len(flux) < 3:
        return {'von_neumann_eta': 0, 'mean_deriv1': 0, 'std_deriv1': 0, 
                'num_peaks': 0, 'period_max_power': 0, 'period_freq_at_max': 0}
    
    eta = np.mean(np.diff(flux)**2) / (np.var(flux) + 1e-9)
    
    dt = np.diff(time)
    mask_dt = dt > 0.001
    if np.any(mask_dt):
        deriv = np.diff(flux)[mask_dt] / dt[mask_dt]
        mean_deriv = np.mean(deriv)
        std_deriv = np.std(deriv)
    else:
        mean_deriv, std_deriv = 0, 0
        
    peaks, _ = find_peaks(flux)
    num_peaks = len(peaks)
    
    max_power, freq_at_max = 0, 0
    try:
        freqs = np.linspace(0.01, 5, 30)
        pgram = lombscargle(time, flux, freqs * 2 * np.pi, normalize=True)
        max_power = np.max(pgram)
        freq_at_max = freqs[np.argmax(pgram)]
    except: 
        pass
    
    return {
        'von_neumann_eta': eta,
        'mean_deriv1': mean_deriv,
        'std_deriv1': std_deriv,
        'num_peaks': num_peaks,
        'period_max_power': max_power,
        'period_freq_at_max': freq_at_max
    }

def bazin_func(time, A, B, t0, t_fall, t_rise):
    """Mô hình Bazin function cho Light curve."""
    with np.errstate(over='ignore', invalid='ignore', divide='ignore'):
        val = A * (np.exp(-(time - t0) / t_fall) / (1 + np.exp((time - t0) / t_rise))) + B
    return np.nan_to_num(val)

def calculate_bazin_features(time, flux):
    """Fit Bazin function và trả về tham số."""
    if len(time) < 5: 
        return np.nan, np.nan, np.nan
    try:
        peak_idx = np.argmax(flux)
        p0 = [np.max(flux), np.min(flux), time[peak_idx], 50, 10]
        bounds = ([0, -np.inf, -np.inf, 1e-3, 1e-3], [np.inf, np.inf, np.inf, 1000, 500])
        popt, _ = curve_fit(bazin_func, time, flux, p0=p0, bounds=bounds, maxfev=800)
        
        flux_pred = bazin_func(time, *popt)
        rmse = np.sqrt(np.mean((flux - flux_pred)**2)) / (np.std(flux) + 1e-9)
        
        return popt[4], popt[3], rmse # t_rise, t_fall, rmse
    except:
        return np.nan, np.nan, np.nan

def calculate_decay_slope(time, flux, time_peak):
    """Tính độ dốc giai đoạn giảm sáng (post-peak)."""
    mask = (time > time_peak) & (flux > 0)
    t_post = time[mask]
    f_post = flux[mask]
    if len(t_post) < 3: return np.nan
    try:
        dt = t_post - time_peak + 1
        slope, _, _, _, _ = linregress(np.log(dt), np.log(f_post))
        return -slope
    except: return np.nan

def calculate_stetson_j_k(flux, flux_err):
    """Tính chỉ số Stetson J và K."""
    n = len(flux)
    if n < 2: return 0, 0
    mean_f = np.mean(flux)
    delta = np.sqrt(n / (n - 1)) * (flux - mean_f) / (flux_err + 1e-9)
    K = (1/np.sqrt(n)) * np.sum(np.abs(delta)) / np.sqrt(np.sum(delta**2))
    P_k = delta**2 - 1
    J = (1/n) * np.sum(np.sign(P_k) * np.sqrt(np.abs(P_k)))
    return J, K

def exponential_func(time, A, t0, tau):
    """Mô hình phân rã mũ (Exponential Decay)."""
    with np.errstate(over='ignore', invalid='ignore'):
        val = np.where(time > t0, A * np.exp(-(time - t0) / (tau + 1e-5)), 0)
    return np.nan_to_num(val)

def calculate_model_comparison(time, flux, bazin_rmse):
    """So sánh sai số giữa mô hình Bazin (đặc trưng TDE) và Exponential (đặc trưng SN)."""
    if len(time) < 5: return np.nan
    try:
        peak_idx = np.argmax(flux)
        p0 = [np.max(flux), time[peak_idx], 20]
        bounds = ([0, -np.inf, 1e-3], [np.inf, np.inf, 500])
        
        popt, _ = curve_fit(exponential_func, time, flux, p0=p0, bounds=bounds, maxfev=800)
        flux_pred = exponential_func(time, *popt)
        sn_rmse = np.sqrt(np.mean((flux - flux_pred)**2)) / (np.std(flux) + 1e-9)
        
        return bazin_rmse / (sn_rmse + 1e-9)
    except:
        return np.nan

In [15]:
# ==========================================
# TRÍCH XUẤT ĐẶC TRƯNG (FEATURE EXTRACTION)
# ==========================================

def extract_features(df_chunk):
    features_list = []
    grouped = df_chunk.groupby('object_id')
    
    for obj_id, group in grouped:
        group = group.sort_values('mjd')
        
        flux = group['flux'].values
        mjd = group['mjd'].values
        bands = group['band'].values
        flux_err = group['flux_err'].values if 'flux_err' in group.columns else np.ones_like(flux)
        
        # 1. Global Features
        stats = {
            'object_id': obj_id,
            'flux_min': np.min(flux),
            'flux_max': np.max(flux),
            'flux_mean': np.mean(flux),
            'flux_median': np.median(flux),
            'flux_std': np.std(flux),
            'flux_skew': skew(flux),
            'flux_kurtosis': kurtosis(flux),
            'flux_mad': median_abs_deviation(flux),
            'flux_amplitude': (np.max(flux) - np.min(flux))/2,
            'duration': mjd[-1] - mjd[0]
        }
        
        try:
            stats['flux_integral_global'] = trapz(y=flux, x=mjd)
        except: 
            stats['flux_integral_global'] = 0

        cusum = np.cumsum(flux - np.mean(flux))
        stats['cusum_range'] = np.max(cusum) - np.min(cusum)
        
        stats.update(calculate_variability_features(flux, mjd))
        
        _, k_global = calculate_stetson_j_k(flux, flux_err)
        stats['stetson_k'] = k_global

        # Weighted Time Statistics
        pos_mask = flux > 0
        if np.sum(pos_mask) > 2:
            t_w = mjd[pos_mask]; f_w = flux[pos_mask]
            w_mean = np.average(t_w, weights=f_w)
            w_std = np.sqrt(np.average((t_w - w_mean)**2, weights=f_w))
            stats['time_width_weighted'] = w_std
            stats['time_skew_weighted'] = np.average(((t_w - w_mean)/(w_std+1e-9))**3, weights=f_w)
        else:
            stats['time_width_weighted'] = 0; stats['time_skew_weighted'] = 0

        # Rise/Fall Time
        peak_idx = np.argmax(flux)
        time_peak = mjd[peak_idx]
        stats['rise_time'] = time_peak - mjd[0]
        stats['fall_time'] = mjd[-1] - time_peak
        stats['rise_fall_ratio'] = stats['rise_time'] / (stats['fall_time'] + 1e-9)
        
        # Linear Slopes
        try:
            stats['slope_rise_global'], _, _, _, _ = linregress(mjd[:peak_idx+1], flux[:peak_idx+1]) if peak_idx > 1 else (0,0,0,0,0)
            stats['slope_fall_global'], _, _, _, _ = linregress(mjd[peak_idx:], flux[peak_idx:]) if len(mjd)-peak_idx > 1 else (0,0,0,0,0)
        except: 
            stats['slope_rise_global'] = 0; stats['slope_fall_global'] = 0
            
        stats['percent_time_above_mean'] = np.sum(flux > np.mean(flux)) / len(flux)
        stats['color_slope_g_r'] = calculate_color_slope(mjd, flux, bands)

        # 2. Band-specific Features
        band_max_flux = {} 
        band_max_times = {}

        for b in ['u', 'g', 'r', 'i', 'z']:
            mask = (bands == b)
            
            if np.sum(mask) > 0:
                f_b = flux[mask]; t_b = mjd[mask]; err_b = flux_err[mask]
                
                stats[f'mean_{b}'] = np.mean(f_b)
                stats[f'max_{b}'] = np.max(f_b)
                stats[f'min_{b}'] = np.min(f_b)
                stats[f'std_{b}'] = np.std(f_b)
                stats[f'amp_{b}'] = np.max(f_b) - np.min(f_b)
                
                band_max_flux[b] = np.max(f_b)
                idx_max_b = np.argmax(f_b)
                band_max_times[b] = t_b[idx_max_b]
                
                try:
                    stats[f'integral_{b}'] = trapz(y=f_b, x=t_b)
                except:
                    stats[f'integral_{b}'] = 0
                
                adv_stats = calculate_advanced_stats(f_b, err_b)
                for k, v in adv_stats.items():
                    stats[f'{k}_{b}'] = v
                
                J, K = calculate_stetson_j_k(f_b, err_b)
                stats[f'stetson_J_{b}'] = J
                stats[f'stetson_K_{b}'] = K
                stats[f'von_neumann_{b}'] = np.mean(np.diff(f_b)**2)/(np.var(f_b)+1e-9) if len(f_b)>2 else 0
                
                p05, p95 = np.percentile(f_b, [5, 95]) if len(f_b)>1 else (f_b[0], f_b[0])
                stats[f'p05_{b}'] = p05; stats[f'p95_{b}'] = p95
                stats[f'flux_width_90_{b}'] = p95 - p05
                try: 
                    stats[f'linear_slope_{b}'] = linregress(t_b, f_b)[0]
                except: 
                    stats[f'linear_slope_{b}'] = 0
                
                # Model Fitting (Bazin & Decay)
                if b in ['u', 'g', 'r']:
                    t_rise_bz, t_fall_bz, bazin_rmse = calculate_bazin_features(t_b, f_b)
                    
                    stats[f'bazin_rise_{b}'] = t_rise_bz
                    stats[f'bazin_fall_{b}'] = t_fall_bz
                    stats[f'bazin_rmse_{b}'] = bazin_rmse
                    
                    if not np.isnan(bazin_rmse):
                        stats[f'chi2_ratio_{b}'] = calculate_model_comparison(t_b, f_b, bazin_rmse)
                    else:
                        stats[f'chi2_ratio_{b}'] = np.nan
                    
                    t_peak_b = t_b[np.argmax(f_b)]
                    stats[f'decay_slope_{b}'] = calculate_decay_slope(t_b, f_b, t_peak_b)

            else:
                band_max_flux[b] = 0
                stats[f'integral_{b}'] = 0
                
                miss_cols = ['mean', 'max', 'min', 'std', 'amp', 'iqr', 'variability_index', 
                             'norm_mad', 'excess_variance', 'flux_ratio_95_5', 'beyond_1std',
                             'stetson_J', 'stetson_K', 'von_neumann', 'p05', 'p95', 
                             'flux_width_90', 'linear_slope']
                for c in miss_cols: stats[f'{c}_{b}'] = 0
                
                if b in ['u', 'g', 'r']:
                    stats[f'bazin_rise_{b}'] = np.nan
                    stats[f'bazin_fall_{b}'] = np.nan
                    stats[f'bazin_rmse_{b}'] = np.nan
                    stats[f'decay_slope_{b}'] = np.nan
                    stats[f'chi2_ratio_{b}'] = np.nan

        # 3. Inter-band Features
        stats['color_g_r'] = stats['mean_g'] - stats['mean_r']
        stats['color_u_z'] = stats['mean_u'] - stats['mean_z']
        
        stats['flux_ratio_u_g'] = band_max_flux['u'] / (band_max_flux['g'] + 1e-9)
        stats['flux_ratio_g_r'] = band_max_flux['g'] / (band_max_flux['r'] + 1e-9)
        stats['flux_ratio_u_r'] = band_max_flux['u'] / (band_max_flux['r'] + 1e-9)
        
        lags = calculate_peak_lags(band_max_times)
        stats.update(lags)
        
        features_list.append(stats)
        
    return pd.DataFrame(features_list)

In [16]:
# ==========================================
# POST-PROCESSING: PHYSICS ESTIMATION
# ==========================================

def planck_func(wav_angstrom, T):
    h = 6.626e-34; c = 3.0e8; k = 1.38e-23
    w = wav_angstrom * 1e-10
    if T < 100: T = 100 
    a = 2.0 * h * c**2
    b = (h * c) / (w * k * T)
    with np.errstate(over='ignore', invalid='ignore'):
        val = a / ( (w**5) * (np.exp(b) - 1.0) )
    return np.nan_to_num(val)

def estimate_temperature(row):
    WAVELENGTHS = {'u': 3685, 'g': 4802, 'r': 6231, 'i': 7542, 'z': 8690}
    wavs = []; fluxes = []
    for b, w in WAVELENGTHS.items():
        if row.get(f'max_{b}_corr', 0) > 0:
            wavs.append(w)
            fluxes.append(row[f'max_{b}_corr'])
    if len(fluxes) < 3: return np.nan
    
    wavs = np.array(wavs); fluxes = np.array(fluxes)
    norm_flux = fluxes / np.max(fluxes)
    def fit_func(w, T): return planck_func(w, T) / np.max(planck_func(w, T))
    try:
        popt, _ = curve_fit(fit_func, wavs, norm_flux, p0=[10000], bounds=([1000], [100000]))
        return popt[0]
    except: return np.nan

def add_physics_features(df):
    """Bổ sung các đặc trưng vật lý: Temperature, Radius, Rest-frame metrics."""
    df_out = df.copy()
    if 'Z' not in df_out.columns: df_out['Z'] = -1
    if 'EBV' not in df_out.columns: df_out['EBV'] = 0
    
    # 1. Extinction Correction
    R_lambda = {'u': 4.239, 'g': 3.303, 'r': 2.285, 'i': 1.698, 'z': 1.263}
    for b in ['u', 'g', 'r', 'i', 'z']:
        A_lambda = R_lambda[b] * df_out['EBV'].fillna(0)
        corr = 10 ** (0.4 * A_lambda)
        if f'mean_{b}' in df_out.columns: df_out[f'mean_{b}_corr'] = df_out[f'mean_{b}'] * corr
        if f'max_{b}' in df_out.columns: df_out[f'max_{b}_corr'] = df_out[f'max_{b}'] * corr

    # 2. Temperature Estimation
    print("Calculating blackbody temperature...")
    df_out['temp_blackbody'] = df_out.apply(estimate_temperature, axis=1)

    # 3. Rest-frame adjustment
    z_factor = 1 + df_out['Z'].clip(lower=0)
    for col in ['duration', 'rise_time', 'fall_time', 'bazin_rise_g', 'bazin_fall_g', 'bazin_rise_r', 'bazin_fall_r']:
        if col in df_out.columns:
            df_out[f'{col}_rest'] = df_out[col] / z_factor

    # 4. Absolute Magnitude & Luminosity Proxy
    dist_mod = 5 * np.log10(df_out['Z'].clip(lower=0.001) + 1e-5)
    for b in ['g', 'r']:
        if f'max_{b}_corr' in df_out.columns:
            df_out[f'abs_mag_{b}'] = -2.5*np.log10(df_out[f'max_{b}_corr'].clip(lower=1e-5)) - dist_mod

    # 5. Photospheric Radius Proxy
    if 'abs_mag_g' in df_out.columns and 'temp_blackbody' in df_out.columns:
        df_out['log_radius_photosphere'] = -0.2 * df_out['abs_mag_g'] - 2 * np.log10(df_out['temp_blackbody'].clip(lower=100))
    else:
        df_out['log_radius_photosphere'] = 0

    # 6. Temperature Change Rate
    if 'color_evol_gr' in df_out.columns and 'duration_rest' in df_out.columns:
        df_out['temp_change_rate'] = df_out['color_evol_gr'] / (df_out['duration_rest'] + 1e-5)
    else:
        df_out['temp_change_rate'] = 0

    print("Dropping intermediate columns (Z, EBV)...")
    df_out = df_out.drop(columns=['Z', 'EBV'], errors='ignore')
    
    return df_out

In [17]:
def load_and_process_data():
    print("Starting data loading and processing...")
    all_features = []
    
    # 1. Feature Extraction from splits
    for i in range(1, 21):
        file_path = f"{BASE_PATH}/split_{i:02d}/train_full_lightcurves.csv"
        if os.path.exists(file_path):
            if i % 5 == 0: print(f"Processing split {i:02d}...")
            df = pd.read_csv(file_path)
            df = df.rename(columns={'Flux': 'flux', 'Flux_err': 'flux_err', 'Filter': 'band', 'Time (MJD)': 'mjd'})
            feats = extract_features(df)
            all_features.append(feats)
    
    full_features = pd.concat(all_features, ignore_index=True)
    
    # 2. Merge Metadata
    print("Merging with metadata...")
    train_log = pd.read_csv(TRAIN_LOG_PATH)
    meta_cols = ['object_id', 'target', 'Z', 'EBV'] 
    available_cols = [c for c in meta_cols if c in train_log.columns]
    
    final_df = pd.merge(train_log[available_cols], full_features, on='object_id', how='right')
    
    # 3. Final Preprocessing
    if 'Z' in final_df.columns: final_df['Z'] = final_df['Z'].fillna(-1)
    if 'EBV' in final_df.columns: final_df['EBV'] = final_df['EBV'].fillna(0)

    print("Computing physics features...")
    final_df = add_physics_features(final_df)
    
    # Fill NaN
    numeric_cols = final_df.select_dtypes(include=[np.number]).columns
    cols_to_fill = [c for c in numeric_cols if c not in ['target', 'object_id']]
    for col in cols_to_fill:
        if final_df[col].isnull().any():
            final_df[col] = final_df[col].fillna(final_df[col].median())
            
    return final_df

def prepare_data_for_training(df):
    X = df.drop(columns=['object_id', 'target'])
    y = df['target']
    feature_names = X.columns.tolist()
    
    print(f"Total features: {len(feature_names)}")
    print("Applying StandardScaler...")
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=feature_names)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    return X_train, X_val, y_train, y_val, feature_names, scaler

In [18]:
def train_model(X_train, y_train, X_val, y_val):
    print("\nTraining Model...")
    
    scale_pos_weight = 10
    print(f"Using scale_pos_weight: {scale_pos_weight}")
    
    model = xgb.XGBClassifier(
        n_estimators=2000,
        learning_rate=0.01,
        max_depth=4,
        min_child_weight=5,
        gamma=1.5,             
        subsample=0.8,
        colsample_bytree=0.6,
        scale_pos_weight=12,
        
        objective='binary:logistic',
        eval_metric=['auc', 'logloss'],
        tree_method='hist',
        device='cuda',
        random_state=42,
    )
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=200
    )
    
    return model

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns

def optimize_threshold(y_val, y_pred_proba):
    """Tìm ngưỡng (threshold) tối ưu dựa trên F1-Score."""
    print("\nOptimizing Threshold...")
    
    thresholds = np.arange(0.01, 1.00, 0.01)
    f1_scores = []
    
    for t in thresholds:
        y_pred_temp = (y_pred_proba >= t).astype(int)
        f1 = f1_score(y_val, y_pred_temp, pos_label=1)
        f1_scores.append(f1)
        
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    best_f1 = f1_scores[best_idx]
    
    print(f"Best Threshold: {best_threshold:.2f} | F1-Score: {best_f1:.4f}")
    return best_threshold

def analyze_and_filter_features(model, feature_names, threshold=0.003):
    imp_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    
    zero_imp = imp_df[imp_df['Importance'] == 0]
    print(f"Features with 0 importance: {len(zero_imp)}")
    
    print(f"\nTop 10 Important Features:")
    print(imp_df.head(10))
    
    selected_features = imp_df[imp_df['Importance'] >= threshold]['Feature'].tolist()
    print(f"Selected {len(selected_features)}/{len(feature_names)} features.")
    
    return selected_features

def evaluate_model_comprehensive(model, X_val, y_val, feature_names, threshold=0.5):
    print(f"\nEvaluating Model (Threshold = {threshold:.2f})...")
    
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    print("\nClassification Report:")
    print(classification_report(y_val, y_pred, target_names=['Non-TDE', 'TDE']))
    
    feat_imp = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False).head(20)
    
    return feat_imp

In [20]:
def run_iterative_strategy(X_train, y_train, X_val, y_val, X_test_full, feature_names, scaler):
    
    params = {
        'n_estimators': 2000,
        'learning_rate': 0.02, 
        'max_depth': 5,           
        'subsample': 0.8,
        'colsample_bytree': 0.6,
        'gamma': 1.0,             
        'scale_pos_weight': 12,   
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'tree_method': 'hist',
        'device': 'cuda',         
        'random_state': 42
    }
    
    print("\nStarting Iterative Pseudo-Labeling Pipeline...")

    # 1. Prepare Data
    X_train_df = pd.DataFrame(X_train, columns=feature_names)
    X_val_df = pd.DataFrame(X_val, columns=feature_names)
    X_test_df = pd.DataFrame(X_test_full, columns=feature_names)
    
    # Initial Feature Selection
    sel_model = xgb.XGBClassifier(**{**params, 'n_estimators': 500})
    sel_model.fit(X_train_df, y_train, eval_set=[(X_val_df, y_val)], verbose=False)
    
    imp_df = pd.DataFrame({'feature': feature_names, 'score': sel_model.feature_importances_})
    top_feats = imp_df.sort_values('score', ascending=False).head(80)['feature'].tolist()
    phy_feats = ['rise_time', 'fall_time', 'amp_g', 'amp_r', 'duration', 'temp_blackbody', 'chi2_ratio_g', 'bazin_rise_g', 'bazin_fall_g']
    final_feats = list(set(top_feats + [f for f in phy_feats if f in feature_names]))
    
    print(f"Selected {len(final_feats)} features for loop.")
    
    X_tr_sub = X_train_df[final_feats]
    X_va_sub = X_val_df[final_feats]
    X_te_sub = X_test_df[final_feats]
    
    X_curr = pd.concat([X_tr_sub, X_va_sub], axis=0)
    y_curr = np.concatenate([y_train, y_val])
    
    # 2. Iterative Loops
    pseudo_thresholds = [0.95, 0.90] 
    
    for i, thresh in enumerate(pseudo_thresholds):
        print(f"Loop {i+1}: Pseudo-Labeling (Threshold > {thresh})")
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_curr, y_curr, verbose=False)
        
        probs = model.predict_proba(X_te_sub)[:, 1]
        idx_new = np.where(probs > thresh)[0]
        print(f"Found {len(idx_new)} new pseudo-labels.")
        
        if len(idx_new) > 0:
            X_pseudo = X_te_sub.iloc[idx_new]
            y_pseudo = np.ones(len(idx_new))
            X_curr = pd.concat([X_curr, X_pseudo], axis=0)
            y_curr = np.concatenate([y_curr, y_pseudo])
        else:
            print("No new samples found. Breaking loop.")
            break

    # 3. Final Averaging
    print("\nFinal Loop: Seed Averaging (5 seeds)")
    seeds = [42, 2024, 888, 77, 123]
    avg_preds = np.zeros(len(X_te_sub))
    final_params = params.copy()
    final_params['n_estimators'] = 3000 
    
    for s in seeds:
        final_params['random_state'] = s
        clf = xgb.XGBClassifier(**final_params)
        clf.fit(X_curr, y_curr, verbose=False)
        avg_preds += clf.predict_proba(X_te_sub)[:, 1]
    
    avg_preds /= len(seeds)
    print("Pipeline completed.")
    
    return avg_preds

In [21]:
def prepare_test_features(save_path="test_features_processed.csv"):
    if os.path.exists(save_path):
        print(f"Loading processed file: {save_path}")
        return pd.read_csv(save_path)

    print("Extracting test features from raw files...")
    aggregated_test = []
    
    for i in range(1, 21):
        file_path = f"{BASE_PATH}/split_{i:02d}/test_full_lightcurves.csv"
        if os.path.exists(file_path):
            if i % 5 == 0: print(f"Processing split {i:02d}...")
            df_chunk = pd.read_csv(file_path)
            df_chunk = df_chunk.rename(columns={'Flux': 'flux', 'Flux_err': 'flux_err', 'Filter': 'band', 'Time (MJD)': 'mjd'})
            feats = extract_features(df_chunk)
            aggregated_test.append(feats)

    if not aggregated_test:
        print("Error: No test data found.")
        return None

    test_features = pd.concat(aggregated_test, ignore_index=True)
    
    print("Merging metadata...")
    test_log_path = f"{BASE_PATH}/test_log.csv"
    if os.path.exists(test_log_path):
        test_log = pd.read_csv(test_log_path)
        meta_cols = ['object_id', 'Z', 'EBV'] 
        available = [c for c in meta_cols if c in test_log.columns]
        final_df = pd.merge(test_log[available], test_features, on='object_id', how='right')
    else:
        final_df = test_features
        final_df['Z'] = -1; final_df['EBV'] = 0

    if 'Z' in final_df.columns: final_df['Z'] = final_df['Z'].fillna(-1)
    if 'EBV' in final_df.columns: final_df['EBV'] = final_df['EBV'].fillna(0)
    
    num_cols = final_df.select_dtypes(include=[np.number]).columns
    cols_to_fill = [c for c in num_cols if c not in ['object_id', 'Z', 'EBV']]
    for c in cols_to_fill:
        if final_df[c].isnull().any(): final_df[c] = final_df[c].fillna(0)

    print("Adding physics features...")
    final_df = add_physics_features(final_df)
    
    final_df.to_csv(save_path, index=False)
    print("Test data preparation complete.")
    return final_df

In [22]:
def predict_and_submit(model, test_df, feature_names, scaler, threshold=0.5, filename="submission.csv"):
    print(f"Generating submission (Threshold={threshold})...")
    
    model_features = getattr(model, 'feature_names_in_', feature_names)
    scaler_features = getattr(scaler, 'feature_names_in_', feature_names)

    df_for_scaler = test_df.copy()
    missing_scaler = set(scaler_features) - set(df_for_scaler.columns)
    for c in missing_scaler: df_for_scaler[c] = 0
            
    try:
        X_full_raw = df_for_scaler[scaler_features]
    except KeyError as e:
        print(f"Error: Missing features for scaler: {e}")
        return

    X_full_scaled_np = scaler.transform(X_full_raw)
    X_full_scaled_df = pd.DataFrame(X_full_scaled_np, columns=scaler_features)
    X_final = X_full_scaled_df[model_features]

    y_prob = model.predict_proba(X_final)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)

    submission = pd.DataFrame({'object_id': test_df['object_id'], 'target': y_pred})
    n_tde = submission['target'].sum()
    print(f"Result: {n_tde} TDEs / {len(submission)} ({n_tde/len(submission):.2%})")
    
    submission.to_csv(filename, index=False)
    print(f"Saved: {filename}")

In [23]:
if __name__ == "__main__":
    # Load & Prepare Data
    df = load_and_process_data()
    X_train, X_val, y_train, y_val, feat_names, scaler = prepare_data_for_training(df)

Starting data loading and processing...
Processing split 05...
Processing split 10...
Processing split 15...
Processing split 20...
Merging with metadata...
Computing physics features...
Calculating blackbody temperature...
Dropping intermediate columns (Z, EBV)...
Total features: 168
Applying StandardScaler...


In [24]:
if __name__ == "__main__":
    # Load processed test data
    test_full_df = prepare_test_features("test_features_processed_v4.csv")
    
    if hasattr(scaler, 'feature_names_in_'):
        scaler_feats = scaler.feature_names_in_
    else:
        scaler_feats = feat_names
    
    df_test_aligned = test_full_df.copy()
    for c in (set(scaler_feats) - set(df_test_aligned.columns)): 
        df_test_aligned[c] = 0
        
    X_test_scaled_np = scaler.transform(df_test_aligned[scaler_feats])
    
    # Run iterative pipeline
    final_probs = run_iterative_strategy(
        X_train, y_train, X_val, y_val, 
        X_test_scaled_np, 
        scaler_feats, 
        scaler
    )

Extracting test features from raw files...
Processing split 05...
Processing split 10...
Processing split 15...
Processing split 20...
Merging metadata...
Adding physics features...
Calculating blackbody temperature...
Dropping intermediate columns (Z, EBV)...
Test data preparation complete.

Starting Iterative Pseudo-Labeling Pipeline...
Selected 84 features for loop.
Loop 1: Pseudo-Labeling (Threshold > 0.95)
Found 25 new pseudo-labels.
Loop 2: Pseudo-Labeling (Threshold > 0.9)
Found 71 new pseudo-labels.

Final Loop: Seed Averaging (5 seeds)
Pipeline completed.


In [25]:
if 'final_probs' in locals():
    print("Processing final probabilities...")
    
    raw_test_df = pd.read_csv("test_features_processed_v4.csv")
    thresholds = [0.18, 0.20, 0.22, 0.25, 0.30]
    
    print("\nGenerating final submissions (Relaxed Physics Filter)...")
    
    for t in thresholds:
        preds = (final_probs >= t).astype(int)
        
        # Filter Logic: Only remove extreme outliers (Rise Time > 200 days rest-frame)
        if 'bazin_rise_g' in raw_test_df.columns:
            mask_bad_rise = (preds == 1) & (raw_test_df['bazin_rise_g'] > 200)
            preds[mask_bad_rise] = 0
            if mask_bad_rise.sum() > 0:
                print(f"[Thresh {t}] Removed {mask_bad_rise.sum()} outliers.")
        
        filename = f"submission_final_relax_{t:.2f}.csv"
        sub = pd.DataFrame({'object_id': raw_test_df['object_id'], 'target': preds})
        sub.to_csv(filename, index=False)
        
        cnt = sub['target'].sum()
        print(f"File: {filename} | TDEs: {cnt} ({cnt/len(sub):.2%})")

Processing final probabilities...

Generating final submissions (Relaxed Physics Filter)...
[Thresh 0.18] Removed 26 outliers.
File: submission_final_relax_0.18.csv | TDEs: 530 (7.43%)
[Thresh 0.2] Removed 23 outliers.
File: submission_final_relax_0.20.csv | TDEs: 509 (7.13%)
[Thresh 0.22] Removed 20 outliers.
File: submission_final_relax_0.22.csv | TDEs: 492 (6.90%)
[Thresh 0.25] Removed 18 outliers.
File: submission_final_relax_0.25.csv | TDEs: 462 (6.48%)
[Thresh 0.3] Removed 16 outliers.
File: submission_final_relax_0.30.csv | TDEs: 425 (5.96%)
